## code 1

In [1]:
import time
import os
import sys

import pybullet as p
#from manipulatorsMOD import SawyerMOD  

from cairo_simulator.core.utils import ASSETS_PATH
from cairo_simulator.core.log import Logger
from cairo_simulator.core.link import *
from cairo_simulator.core.simulator import Simulator, SimObject
from cairo_simulator.devices.manipulators import Sawyer
from cairo_simulator.devices.manipulatorsMOD import SawyerMOD #New python file that allows a modified sawyer robot to be used (ie, a sawyer holding a block)

from utils import load_configuration, save_config_to_configuration_file, manual_control, create_cuboid_obstacle

from pybullet_utils import bullet_client as bc
from pybullet_utils import urdfEditor as ed
import pybullet_data
import time



def main():

    use_ros = False
    use_real_time = True
    logger = Logger()
    sim = Simulator(logger=logger, use_ros=use_ros, use_real_time=use_real_time) # Initialize the Simulator
    p.setGravity(0,0,-9.81)
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,0)
    p.setPhysicsEngineParameter(enableFileCaching=0)
    ground_plane = SimObject("Ground", "plane.urdf", [0,0,0])

    table = SimObject('table', 'NEWtable.urdf',  (0.9, 0.1, .47),(1.5708*2,0,0),fixed_base=1)
    #print(p.getNumJoints(table.get_simulator_id()))
    
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
    
    block = SimObject('block1', '1block.urdf', (0.69, -0.2, .55))
    sim_obj1 = SimObject('hole1', '1.5hole.urdf',  (0.69, 0.1, .530),(0,0,0),fixed_base=0)  #1.5708 for 90 deg rotation
    sim_obj2 = SimObject('hole2', '1.25hole.urdf', (0.69, 0.3, .530),(0,0,0),fixed_base=0)
    sim_obj3 = SimObject('hole3', '1.15hole.urdf', (0.69, 0.5, .530),(0,0,0),fixed_base=0)    
    
    #___________________#Test Combining URDF's
    #https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p0 = bc.BulletClient(connection_mode=p.DIRECT)
    p0.setAdditionalSearchPath(pybullet_data.getDataPath())
    p1 = bc.BulletClient(connection_mode=p.DIRECT)
    p1.setAdditionalSearchPath(pybullet_data.getDataPath())

    kuka = p0.loadURDF('1.5hole.urdf')
    #husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)  #this flag appears to have no impact
    husky = p1.loadURDF('1block.urdf')
    
    ed0 = ed.UrdfEditor()
    ed0.initializeFromBulletBody(husky, p1._client)
    ed1 = ed.UrdfEditor()
    ed1.initializeFromBulletBody(kuka, p0._client)

    parentLinkIndex = 0

    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]

    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    #link index = joint index
    newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
    newjoint.joint_type = p0.JOINT_FIXED

    ed0.saveUrdf("combined.urdf")
    sim_obj3 = SimObject('hole3',"combined.urdf", (0.69, -0.4, .530),(0,0,0),fixed_base=0)  
    
    #_______________________ end URDF combining test
    
    
    #print(ASSETS_PATH)
    robotfilelocation=ASSETS_PATH + 'sawyer_description/urdf/sawyer_static.urdf'
    #sawyer_robot = Sawyer("sawyer0", [0, 0, 0.8], fixed_base=1)
    #sawyer_robot = SawyerMOD("sawyer0",robotfilelocation, [0, 0, 0.8], fixed_base=1)
    
    #print(sawyer_robot._arm_dof_indices) #[3, 8, 9, 10, 11, 13, 16]
    #print(sawyer_robot._arm_dof_names) 
    #['right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6']
    
    
    #___________________#Modifying Sawyer URDF's
    #edited from https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p3 = bc.BulletClient(connection_mode=p.DIRECT)
    p3.setAdditionalSearchPath(pybullet_data.getDataPath())
    p4 = bc.BulletClient(connection_mode=p.DIRECT)
    p4.setAdditionalSearchPath(pybullet_data.getDataPath())

    child = p3.loadURDF('1block.urdf')
    parent = p4.loadURDF(robotfilelocation)
    
    ed_child = ed.UrdfEditor()
    ed_child.initializeFromBulletBody(child, p3._client)
    ed_parent = ed.UrdfEditor()
    ed_parent.initializeFromBulletBody(parent, p4._client)
    
    #link index = joint index
   
    parentLinkIndex2 =16

    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]

    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    newjoint2 = ed_parent.joinUrdf(ed_child, parentLinkIndex2, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p3._client, p4._client)
    newjoint2.joint_type = p3.JOINT_FIXED

    ed_parent.saveUrdf("combinedRobot.urdf")

    robotfilelocation2='/home/scott/ScheragaThesis/combinedRobot.urdf'
    
    sawyer_robot = SawyerMOD("sawyer0",robotfilelocation, [0, 0, 0.8], fixed_base=1)
    robotID=sawyer_robot.get_simulator_id() #numeric code for robot
    #print(robotID)  #robot id is 7
    
    #print(get_joint_info_by_name(7,"right_j6"))
    """
    dofindex=[3, 8, 9, 10, 11, 13, 16]
    for x in dofindex:
        print(get_joint_info(7,x))
        print("  ")
    """
    #print(sawyer_robot._arm_dof_indices) #[3, 8, 9, 10, 11, 13, 16]
    #print(sawyer_robot._arm_dof_names) 
    #['right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6']
    
    #get_joint_info_by_name(body, name):
    
#     for j in range(p.getNumJoints(sawyer_robot, physicsClientId=self.id)):
#             joint_info = p.getJointInfo(sawyer_robot, j, physicsClientId=self.id)
#             print(joint_info)
            
            #joint_name = joint_info[1]
            #joint_pos = joint_positions[j]
            #lower_limit = joint_info[8]
            #upper_limit = joint_info[9]
   
    #_______________________
    
    
    
    dofindex=[3, 8, 9, 10, 11, 13, 16]
    for x in dofindex:
        p.enableJointForceTorqueSensor(7,x,enableSensor=1)  #args: bodyID#, jointIndex,enablesensor
    
    
    #print(sawyer_robot._arm_dof_indices) #[3, 8, 9, 10, 11, 13, 16]
    #joint16info=p.getJointState(7,16) 
    
    #print("joint16 state")
    #print(joint16info)#joint16info= jointPosition,jointVelocity,(6dof)jointReactionForces, appliedJointMotorTorque
    #print(" ")
    
    orientation= [0,1,0,0] #quaternion hand pointing down..ish
    xoffset=0
    yoffset=0
    
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 1]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    sawyer_robot.move_to_joint_pos(joint_config)
    time.sleep(1.)
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 0.8]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    sawyer_robot.move_to_joint_pos(joint_config)
    
    key_w = ord('u') 
    key_a = ord('h') 
    key_s = ord('j') 
    key_d = ord('k') 
    
    key_o = ord('o') #z up
    key_l = ord('l') #z down
    key_7 = ord('7') #print all 6dof force/torque info
    key_8 = ord('8') #print info
    
    loop_count = 0 # Loop until someone shuts us down
    
    xcurrent=0.69
    ycurrent=0.1
    zcurrent=0.8
    # the center of 1.5 hole is roughly 0.71  0.08 OR   ~0.78 in x and y
    #print("jointPosition,jointVelocity,(6dof)jointReactionForces, appliedJointMotorTorque")
    while True:
            xoffset=0
            yoffset=0
            
            keys = p.getKeyboardEvents()
            if key_w in keys and keys[key_w] & p.KEY_WAS_TRIGGERED:
                ycurrent+=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_a in keys and keys[key_a] & p.KEY_WAS_TRIGGERED:
                xcurrent-=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_s in keys and keys[key_s] & p.KEY_WAS_TRIGGERED:
                ycurrent-=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_d in keys and keys[key_d] & p.KEY_WAS_TRIGGERED:
                xcurrent+=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_o in keys and keys[key_o] & p.KEY_WAS_TRIGGERED:
                zcurrent+=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_l in keys and keys[key_l] & p.KEY_WAS_TRIGGERED:
                zcurrent-=0.01
                joint_configuration = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_configuration,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)    
                
            if key_7 in keys and keys[key_7] & p.KEY_WAS_TRIGGERED:
                print(" ")
                dofindex=[3, 8, 9, 10, 11, 13, 16]
                print("JointReactionForces")
                for x in dofindex:
                    jointinfo=p.getJointState(7,int(x)) 
                    rounded6dof = [round(num, 3) for num in jointinfo[2]]
                    print("J", x , " ", rounded6dof)
                    
               
                
                
            if key_8 in keys and keys[key_8] & p.KEY_WAS_TRIGGERED:
                print(" ")
                print("location", xcurrent,"  ",ycurrent,"  ",zcurrent)
                #print("enter joint number [3, 8, 9, 10, 11, 13, 16]")
                #a = input()
                #jointinfo=p.getJointState(7,16) 
                #jointinfo=p.getJointState(7,int(a))
                jointinfo=p.getJointState(7,16) 
                rounded6dof = [round(num, 3) for num in jointinfo[2]]
                print("J 16",rounded6dof)
                #print(jointinfo)
                
            time.sleep(0.2)
            
            
            
            
    #manual_control(sawyer_robot)    
    try:
        while True:
            sim.step()
    except KeyboardInterrupt:
        p.disconnect()
        sys.exit(0)

In [ ]:
main()


7
jointPosition,jointVelocity,(6dof)jointReactionForces, appliedJointMotorTorque
 
location 0.69    0.1    0.8
J 16 [-0.478, 0.731, -17.668, 0.082, -0.017, -0.004]
 
location 0.6399999999999999    0.20000000000000007    0.8
J 16 [-0.347, 0.679, -17.673, 0.084, -0.012, -0.003]
 
JointReactionForces
J 3   [-0.008, -0.008, 225.499, 0.326, -63.262, 0.0]
J 8   [-138.503, 52.135, -0.006, 6.633, 17.624, 3.874]
J 9   [-53.661, -80.973, -36.572, 26.931, -8.065, 0.81]
J 10   [-20.49, 50.341, -67.768, 19.877, 5.92, 0.817]
J 11   [50.895, -3.525, -36.274, -2.867, 6.67, 0.38]
J 13   [0.754, 52.274, -2.949, -0.606, 0.019, -0.343]
J 16   [-0.187, 1.011, -17.66, 0.071, -0.005, -0.002]
 
location 0.72    0.20000000000000007    0.75
J 16 [-0.166, 0.979, -17.662, 0.072, -0.001, -0.001]
 
location 0.72    0.20000000000000007    0.6799999999999999
J 16 [-2.699, -130.123, 684.473, 21.276, -33.184, -0.024]
 
JointReactionForces
J 3   [-56.946, -137.734, -371.125, 32.104, 383.949, 2.731]
J 8   [410.958, -188.

In [5]:
#tuple1 = ("abc", 34, True, 40, "male")
tuple2 =(-1.0385539160942905, -1.9923462796223035e-08, 
         (-49.22967538289247, -83.59641821651995, -36.899233075760876,24.7201743877905, -4.291147134456251, 0.7419092278209636), 
         -24.009962659594642)
print(tuple2)
print(tuple2[2])
print(tuple2[2])
round_to_tenths = [round(num, 3) for num in tuple2[2]]
print(round_to_tenths)

(-1.0385539160942905, -1.9923462796223035e-08, (-49.22967538289247, -83.59641821651995, -36.899233075760876, 24.7201743877905, -4.291147134456251, 0.7419092278209636), -24.009962659594642)
(-49.22967538289247, -83.59641821651995, -36.899233075760876, 24.7201743877905, -4.291147134456251, 0.7419092278209636)
(-49.22967538289247, -83.59641821651995, -36.899233075760876, 24.7201743877905, -4.291147134456251, 0.7419092278209636)
[-49.23, -83.596, -36.899, 24.72, -4.291, 0.742]


In [ ]:
#Force torque sensor:
#C code:
#b3CreateSensorEnable6DofJointForceTorqueSensor

#in the quickstart guide : enableJointForceTorqueSensor
"""
https://pybullet.org/Bullet/phpBB3/viewtopic.php?t=12448
    
p.enableJointForceTorqueSensor(block,0,enableSensor=1)  #args: bodyID#, jointIndex,enablesensor

dum2=p.getJointState(block,0)

"""

## code 2


In [ ]:
    #___________________#Combining URDF's
    #https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p0 = bc.BulletClient(connection_mode=p.DIRECT)
    p0.setAdditionalSearchPath(pybullet_data.getDataPath())
    p1 = bc.BulletClient(connection_mode=p.DIRECT)
    p1.setAdditionalSearchPath(pybullet_data.getDataPath())

    #husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)  #this flag appears to have no impact
    husky = p1.loadURDF('1block.urdf')
    kuka = p0.loadURDF('1.5hole.urdf')

    ed0 = ed.UrdfEditor()
    ed0.initializeFromBulletBody(husky, p1._client)
    ed1 = ed.UrdfEditor()
    ed1.initializeFromBulletBody(kuka, p0._client)

    parentLinkIndex = 0

    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]

    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
    newjoint.joint_type = p0.JOINT_FIXED

    ed0.saveUrdf("combined.urdf")
    #pgui = bc.BulletClient(connection_mode=p.GUI)
    #pgui.configureDebugVisualizer(pgui.COV_ENABLE_RENDERING, 0)
    
    orn = [0, 0, 0, 1]
    #ed0.createMultiBody([0, 0, 0], orn, pgui._client)  #doesn't work
    
    
    #ed0.createMultiBody([(0.69, -0.1, .530)], orn)  #works
    sim_obj3 = SimObject('hole3',"combined.urdf", (0.69, -0.4, .530),(0,0,0),fixed_base=0) 


0
1
p0.getNumBodies()= 1
p1.getNumBodies()= 1
fileNames= ['./1.5in squarehole_C.SLDPRT/meshes/1.5in squarehole_C.SLDPRT.STL']


 aaa


ehllo aaa
